In [ ]:
import torch
import cv2
import tqdm
from PIL import Image
import pandas as pd
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

#Строим нейросеть#


Создаем даталоадеры для обучения нейросети

In [ ]:
images_dataset = torchvision.datasets.ImageFolder("./drive/MyDrive/ML/rucode_house/data/train1", transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
    ]))
images_dataloader = torch.utils.data.DataLoader(images_dataset, batch_size=16, shuffle=True)

Будем использовать предобученную на ImageNet сеть ResNet18

In [ ]:
net = models.resnet18(True, True).cuda()
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Заменим последний полносвязный слой сети на слой, который будет выдавать 4 значения на выходе (т.к. у нас 4 класса)

In [ ]:
net.fc = nn.Linear(512,4)

Заморозим все слои нейросети, кроме самого последнего, только что добавленного fc-слоя. Будем обучать только последний слой сети.

In [ ]:
for i, child in enumerate(net.children()):
  if i==9:
    break
  for param in child.parameters():
    param.requires_grad = False

Объявляем лосс-функцию и оптимизатор:

In [ ]:
# стандартная лосс-функция для задачи классификации
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# для обучения на GPU
device = 'cuda:0'
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Обучаем сеть 5 эпох:

In [ ]:
n_epochs = 5
print_every = 10

total_step = len(images_dataloader)

for epoch in range(1, n_epochs+1):
  print(f'Epoch {epoch}\n')
  for batch_idx, (data, target) in enumerate(images_dataloader):
    # кладем данные на GPU
    data, target = data.to(device), target.to(device)

    # делаем шаг обучения сети
    optimizer.zero_grad()
    outputs = net(data)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    if (batch_idx) % 20 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, n_epochs, batch_idx, total_step, loss.item()))

Epoch 1



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/5], Step [0/148], Loss: 1.6830
Epoch [1/5], Step [20/148], Loss: 0.8080
Epoch [1/5], Step [40/148], Loss: 0.6496
Epoch [1/5], Step [60/148], Loss: 0.4331
Epoch [1/5], Step [80/148], Loss: 0.4345
Epoch [1/5], Step [100/148], Loss: 0.3975
Epoch [1/5], Step [120/148], Loss: 0.4259
Epoch [1/5], Step [140/148], Loss: 0.4603
Epoch 2

Epoch [2/5], Step [0/148], Loss: 0.2872
Epoch [2/5], Step [20/148], Loss: 0.6118
Epoch [2/5], Step [40/148], Loss: 0.2406
Epoch [2/5], Step [60/148], Loss: 0.2860
Epoch [2/5], Step [80/148], Loss: 0.2225
Epoch [2/5], Step [100/148], Loss: 0.3495
Epoch [2/5], Step [120/148], Loss: 0.2826
Epoch [2/5], Step [140/148], Loss: 0.1784
Epoch 3

Epoch [3/5], Step [0/148], Loss: 0.1778
Epoch [3/5], Step [20/148], Loss: 0.1165
Epoch [3/5], Step [40/148], Loss: 0.3383
Epoch [3/5], Step [60/148], Loss: 0.0799
Epoch [3/5], Step [80/148], Loss: 0.7694
Epoch [3/5], Step [100/148], Loss: 0.2890
Epoch [3/5], Step [120/148], Loss: 0.2206
Epoch [3/5], Step [140/148], Loss:

Тестируем обученную сеть на тестовом наборе картинок

In [ ]:
#image_path=f'./drive/MyDrive/ML/rucode_house/data/test/2381.jpg'
#image = cv2.imread(image_path)
transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
])
#net.eval()
#predict = net(image)


# load your image(s)
img = Image.open(f'./drive/MyDrive/ML/rucode_house/data/test/2381.jpg')

# Transform
input = transform(img)

# unsqueeze batch dimension, in case you are dealing with a single image
input = input.unsqueeze(0)

input = input.to(device)

# Set model to eval
net.eval()

# Get prediction
output = net(input)

prediction = int(torch.max(output.cpu().data, 1)[1].numpy())
prediction

1

In [ ]:
result = []
transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
])
net.eval()
for name in range(2289,4734):
  image_path=f'./drive/MyDrive/ML/rucode_house/data/test/{name}.jpg'
  try:
    img = Image.open(image_path)
  except:
    continue
  input = transform(img)
  input = input.unsqueeze(0)
  input = input.to(device)
  output = net(input)
  prediction = int(torch.max(output.cpu().data, 1)[1].numpy()) 
  result.append(prediction)


In [ ]:
submission = pd.DataFrame(result)
submission.to_csv('ans.csv', index=False,header=False)

In [ ]:
datas = pd.read_csv("./ans.csv")
datas.head()

,0
0,0
1,0
2,0
3,0
4,0
